In [1]:
API_KEY = 'a6af2051-a60d-4d5b-ae19-994ff9f22f24_up9205w'

In [27]:
import requests
import json

class Smartlead:
    BASE_URL = 'https://server.smartlead.ai/api/v1'

    def __init__(self, api_key):
        self.api_key = api_key

    # add create campaign method

    def get_campaign_sequences(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/sequences?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign_email_accounts(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/email-accounts?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign_leads(self, campaign_id, offset=0, limit=10):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/leads?api_key={self.api_key}&offset={offset}&limit={limit}"
        response = requests.get(url)
        return response.json()
    
    def post_campaign_leads(self, campaign_id, lead_list):
        """`lead_list` format is 
            
        lead_list = [
            {
                "first_name": "Cristiano",
                "last_name": "Ronaldo",
                "email": "cristiano@mufc.com",
                "phone_number": "0239392029",
                "company_name": "Manchester United",
                "website": "mufc.com",
                "location": "London",
                "custom_fields": {
                    "Title": "Regional Manager",
                    "First_Line": "Loved your recent post about remote work on Linkedin"
                },
                "linkedin_profile": "http://www.linkedin.com/in/cristianoronaldo",
                "company_url": "mufc.com"
            }
            # ... (add up to 100 leads)
        ]
        """
        settings = {
            "ignore_global_block_list": True,
            "ignore_unsubscribe_list": True,
            "ignore_duplicate_leads_in_other_campaign": False
        }
        if not isinstance(lead_list, list) or not isinstance(settings, dict):
            raise ValueError("lead_list must be a list and settings must be a dictionary.")

        if len(lead_list) > 100:
            raise ValueError("You can only send a maximum of 100 leads at a time.")

        url = f"{self.BASE_URL}/campaigns/{campaign_id}/leads?api_key={self.api_key}"
        data = {
            "lead_list": lead_list,
            "settings": settings
        }
        headers = {'Content-Type': 'application/json'}
        response = requests.post(url, headers=headers, data=json.dumps(data))
        return response.json()

# Example usage:
api_key = API_KEY
campaign_id = '123383'

smartlead = Smartlead(api_key)
sequences = smartlead.get_campaign_sequences(campaign_id)
campaign = smartlead.get_campaign(campaign_id)
email_accounts = smartlead.get_campaign_email_accounts(campaign_id)
leads = smartlead.get_campaign_leads(campaign_id, offset=0, limit=10)

print(sequences)
print(campaign)
print(email_accounts)
print(leads)


[{'id': 387060, 'created_at': '2023-10-27T18:11:13.050Z', 'updated_at': '2023-10-27T18:11:13.050Z', 'email_campaign_id': 123383, 'seq_number': 1, 'seq_delay_details': {'delayInDays': 1}, 'subject': '{{subject_line}}', 'email_body': '<div>{{body}}&nbsp;</div>', 'sequence_variants': None}]
{'id': 123383, 'user_id': 14098, 'created_at': '2023-10-27T18:11:00.932Z', 'updated_at': '2023-11-07T05:05:09.172Z', 'status': 'ARCHIVED', 'name': 'Jennifer / Worley - Test Campaign', 'track_settings': [], 'scheduler_cron_value': {'tz': 'America/Los_Angeles', 'days': [1, 2, 3, 4, 5], 'endHour': '18:00', 'startHour': '09:00'}, 'min_time_btwn_emails': 20, 'max_leads_per_day': 100, 'stop_lead_settings': 'REPLY_TO_AN_EMAIL', 'enable_ai_esp_matching': False, 'send_as_plain_text': False, 'follow_up_percentage': 100, 'unsubscribe_text': None, 'client_id': None}
[{'id': 492739, 'created_at': '2023-10-27T18:03:11.485Z', 'updated_at': '2023-11-07T04:47:11.573Z', 'user_id': 14098, 'from_name': 'Jennifer Lee', 'fr

In [28]:
lead_list = [
    {
        "first_name": "Cristiano",
        "last_name": "Ronaldo",
        "email": "cristiano@mufc.com",
        "phone_number": "0239392029",
        "company_name": "Manchester United",
        "website": "mufc.com",
        "location": "London",
        "custom_fields": {
            "Title": "Regional Manager",
            "First_Line": "Loved your recent post about remote work on Linkedin"
        },
        "linkedin_profile": "http://www.linkedin.com/in/cristianoronaldo",
        "company_url": "mufc.com"
    }
    # ... (add up to 100 leads)
]



response = smartlead.post_campaign_leads(campaign_id, lead_list)
